# Weekly Report - July 4
## Summary of Contents
- Goals
- Results & Findings
- Plan of the Next Week
- Appendix

## Goals
My goals of the week include:
- Implement more metrices to evaluate anomaly detectors
- Anlayze and address the fallacy of the Gaussian-based anomaly detectors
- Assess different configuration of Deep Autoencoders
- Apply Anomaly Detection on Synthetic Datasets

## Results & Findings
### 1. Anomaly Detection on Synthetic Datasets
I created 4 synthetic binary vector datasets. Here is a brief introduction of them:

#### Characteristics of the 4 dataset
|Dataset|#1|#2|#3|#4|
|---|---|---|---|---|
|Sample Size|85,322|100,000|100,000|85,674|
|No. Dimensions|16|16|16|16|
|No of gaussian distribution used to generate the dataset|1|1|3|2|
|Anomaly|the total number of '1's in the vector is less than the threshold (4)|The sum of the right (`n-1`) digits is even **AND** the leftmost digit is even (1)|Generated with two different distribution other than the normal one|the total number of '1's in the vector is less than the threshold (4)|
|%Anomaly|11.8%|13.1%|10.0%|11.2%|

#### Basic Method to Generate Data - Achieve Correlation and Binary
> An example code can be found in the *Appendix A*)

Initially I generated a binary dataset by setting 1/0s randomly in each vectors. Then I found neither PCA nor Deep Autoencoder works in this dataset, because each dimension is independently with each other. Therefore, here is how I did to achieve correlation in the random binary dataset 

- First, I generated a dataset with a multivariate distribution with a random mean vector and a covariance matrix
- Second, I convert the dataset to binary by setting all the numbers that is larger or equal to 0.5 to 1, and others to 0. 

#### Evaluation of the Anomaly Detector
> The detailed matrices of the performance of each detector in the four datasets are recorded in the Appendix B

**In summary, there is no Anomaly Detector that works well in all the four datasets. The "Autoencoder+Reconstruction"-based method performs very similar to the two PCA-based methods. All of them are good at identifying anomalies that are generated with different gaussian distribution than the normal data. In comparison, the "Autoencoder+Gaussian"-based method perform very differently, and it excels when Anomalies are just outliers that come from the same distribution with the normal data.**


Two PCA-based methods (Reconstruction Error and Gaussian) and "Autoencoder+Reconstruction"-based method work the best in dataset 3, where anomaly data points are generated with different gaussian distributions. After encoding/decoding, the anomaly will be ranked in a long tail, which is very ideal for detection.

> Example Plot: Good result of Reconstruction Error-based detector with PCA method on Dataset #3

![PCA_REC_SD3](Screenshots/704_PCA_REC_SD3.png)

<br>

If we compare the reconstruction from the PCA and Autoencoder, we can find the Autoencoder creates a better work that is more similar to the original data.

> Reconstruction from the PCA: the original matrix is binary, so blue cell represents 0 and red cell represents 1. As we can see from the plot, the reconstruction is successful but noisy

![PCA_REC_SD3_PLOT](Screenshots/704_PCA_REC_PLOT_SD3.png)

> Reconstruction from the Autoencoder: in comparison, the reconstruction from the Autoencoder is much cleaner and accurate. 

![AUT_REC_SD3_PLOT](Screenshots/704_AUT_REC_PLOT_SD3.png)


<br>

However, when the anomaly data points are generated with the same gaussian distributions with the normal data points, the PCA-based methods and "Autoencoder+Reconstruction Error" method cannot work well. As we can see from the plot below, the anomaly points scatter across the line, which is difficult for a precise detection. 

> Example Plot: bad result of Reconstruction Error-based detector with PCA method on Dataset #1

![PCA_REC_SD1](Screenshots/704_PCA_REC_SD1.png)

> Example Plot: bad result of Reconstruction Error-based detector with Autoencoder method on Dataset #1

![AUT_REC_SD1](Screenshots/704_AUT_REC_SD1.png)

<br>

In comparison, the "Autoencoder+Gaussian" methods work surprisingly well in the dataset #1 and #4, where the anomalies are generated with the same gaussian distribution(s) with the normal data points, but on the left tail of the distribution curve (the total number of 1s is below the threshold). In both datasets, the method achieves 100% in Precision@50, while other methods get only 10% or less. However, by taking a further look, I noticed that in both datasets, the gaussian detector sets $\lambda = 0$ as the optimal parameter in "whitening" process, which means it uses an identical matrix as the covariance. As you can see in the two line plots below, the matrices will drop significantly if we use a non-zero $\lambda$. This is confusing to me. 

> Example Plot: good result of Gaussian-based detector with Autoencoder method on Dataset #1

![AUT_GAU_SD1](Screenshots/704_AUT_GAU_SD1.png)

> Line plots of R-Precision and Precision@50 achieved with different lambda in "whitening process". lambda=0 is selected as the optimal parameter value in both dataset#1 and #4

![AUT_GAU_SD1_LAMBDA](Screenshots/704_AUT_GAU_SD1_LAMBDA.png)


The last but not least, recall is one of the matrices I used for evaluation. The recall matric in almost all the tests are consistently higher than the precision-related matrices (except the PCA Reconstruction in Dataset 4 and Autoencoder Reconstruction method in Dataset 2. Considering that I used F-score ($F = \frac{2*Precision*Recall}{Precision + Recall}$) as the optimization objective through training, it should be easier to achieve high recall than high precision in these datasets. I will consider to give more weight to precision in the F-score formula for further analysis. 

### 2. Assess different configuration of Deep Autoencoders
I assess the effect of the four factors in Autoencoders on the performance of anomaly detection. The assessment is conducted with the Reconstruction Error-based method on MNIST Hand-written digits data. The four factors includes
- Number of layers
- Layer size (# neurons in the deep net)
- Drop-out rate
- Batch size of training

#### Effect of No. Layers on the Performance
First of all, an autoencoder model includes an encoder model and a decoder model. The number of layers here represents the number of layers in a single encoder model as well as the the single decoder model. 

I first assess the performance of network with 2-5 layers. As the plot below shows, the precision matrices increase with the number of layers. Therefore, I add one more assessment on the 6-9 layers, which is shown with the second plot below. The performance drops after the number of layers exceeds 7. My explanation to the fact is the final encoded data has too few dimension to contain enough information: for every additional layer in the encoder, the layer size is set as a half of the previous layer. Considering the input image has dimension of 1024, the final encoded data after 9 layers will only have a dimension of $1024/2^7 = 8$. I will probably set a lower limit on the layer size for further assessment. For now, it seems a layer of size 5-7 works well.

![704_autoencoder_config_assessment_n_layers](Screenshots/704_autoencoder_config_assessment_n_layers.png)
![704_autoencoder_config_assessment_n_layers2](Screenshots/704_autoencoder_config_assessment_n_layers2.png)

#### Effect of layer size on the Performance
All of the models in the assessment contains 4 layers, and I modified different layer size to assess its effect on the performance. The x-label of the plot below is total number of parameters (nerons) in the entire autoencoder model. 

As shown in the plot below, the performance reaches a peak at around 600,000 parameters. For reference, the number of data points at training is 50,407 (all normal). 

![704_autoencoder_config_assessment_layer_size](Screenshots/704_autoencoder_config_assessment_layer_size.png)

#### Effect of the Drop Out Rate on the Performance
I assess the effect of different drop out rate on a model with 5 layers (1,399,712 parameters). Based on the plot, it shows the performance drops as the drop-out rate increases from 0. However, it worths noting that the precision @ 50 increases when the drop-out rate increases to 10% from 0. I will take a further analysis on the drop-out rate between 0 and 10% to see if there is a benefit to introduce drop-out in the model.

![704_autoencoder_config_assessment_drop_out](Screenshots/704_autoencoder_config_assessment_drop_out.png)

#### Effect of Batch size of training on the Performance
I tested different batch size (256,512,1024, and 2048). In addition to the Precision@k and R-Precision, I also includes the total training time in the matrices. My original hypothesis is the larger the batch size, the better results will be achieved, but it will take longer training time. However, the plot below is against my hypothesis. I will run further experiment on this.
![704_autoencoder_config_assessment_batch_size](Screenshots/704_autoencoder_config_assessment_batch_size.png)

### 3. Analysis on the Gaussian-based Model
#### 2-D Visualization of data after encoding
To visualize a 2D Distribution Density, I used PCA-Encoding and only keep the top 2 principal components of the dataset for assessment. The plots below comes from the Face and MNIST dataset. The yellow points in the plot incidates anomlay, and purple ones are normal points. For every set of visualization, the first plot on the top is the scatterplot of the entire dataset, and the subplots below shows a scatterplot of 500 sample points each. 

As we can see in the plots, the anomlay is embedded in the cluster of the normal data points, so it makes the detection difficult, especially for the gaussian approach.

> Below: 2D Visualization of MNIST Data

![704_2D_VIS_GAU_MNIST](Screenshots/704_2D_VIS_GAU_MNIST.png)

> Below: 2D Visualization of Face Data (This dataset only contains over 600 images, so the second subplot has only over 100 points)

![704_2D_VIS_GAU_FACE](Screenshots/704_2D_VIS_GAU_FACE.png)

<br>

Here I plot a 2D Visualization of the synthetic dataset#1, #2, and #3 (I applied PCA and select the top 2 principal component to encode the data). 

Clearly, in the dataset #1, the distribution of anomaly is concentrated on the right of the data cluster. It is to my surprise that the PCA-Gaussian methods fail in this case (I have double checked that I only use the normal dataset when training the PCA and Autoencoder).

None of the four detectors work well in the dataset#2. For the plot of the dataset #2 before, the anomaly is distributed vertically within the normal dataset (slightly concentrated on the top half of the cluster). It adds diffult to the gaussian-based detector to identify anomalies through low probability.

For the plot of the dataset #3, clearly, the anomalies has distribution that has a more narrow shape than the normal dataset. It makes sense that the PCA-Gaussian method work well in this dataset. 

> Below: 2D Visualization of Synthetic Dataset #1

![704_2D_VIS_GAU_SD1](Screenshots/704_2D_VIS_GAU_SD1.png)

> Below: 2D Visualization of Synthetic Dataset #2

![704_2D_VIS_GAU_SD2](Screenshots/704_2D_VIS_GAU_SD2.png)

> Below: 2D Visualization of Synthetic Dataset #3

![704_2D_VIS_GAU_SD3](Screenshots/704_2D_VIS_GAU_SD3.png)

### 4. Whitening Covariance
In summary, the "whitening" process solves the problem of singularity, but it does not save the fallacy of Autoencoder + Gaussian method in application. 

#### Implementation
A big challenge is to select the optimal $\lambda$. I used cross-validation to split the training set into training and validation sets, and assess 10 $\lambda$ values to find the optimal one. The target of the optimization is F score$F = \frac{2*Precision*Recall}{Precision + Recall}$. I tried setting Precision@k and R-Precision as targets, but that made the model very conservative and sacrifice the recall score. 

#### Results
Below I put the result of after-whitening performance on the MNIST data and Synthetic Dataset. The "whitening" process does solve the problem of singularity, so that the autoencoder-gaussian approach can be applied on the MNIST dataset; however, the precision of the detector is still very low on both the dataset. For all the dataset I have test on, the Autoencoder-gaussian approach only works well on the Synthetic dataset #1 and #4 (as talked above)

> Below: Result of Whitening on the Autoencoder-Gaussian method on MNIST Data

![whitening_1](Screenshots/704_Whitening_1.png)
![whitening_2](Screenshots/704_Whitening_2.png)

> Below: Result of Whitening on the Autoencoder-Gaussian method on Synthetic dataset #2

![whitening_3](Screenshots/704_Whitening_3.png)
![whitening_4](Screenshots/704_Whitening_4.png)

### 4. Diagonal of the Cov matrix
I plot out the covariance matrix of the dataset using a heat map. The sample plots below proves that the covariance matrix is diagonal. 

> Below: Diagonal of the Covariance Matrix from the Face Dataset

![DIAGONAL_FACE](Screenshots/704_DIAGONAL_FACE.png)

> Below: Diagonal of the Covariance Matrix from the Synthetic Dataset #3

![DIAGONAL_SD3](Screenshots/704_DIAGONAL_SD3.png)

> Below: Diagonal of the Covariance Matrix from the MNIST Dataset (There are 200 dimensions, so I plot the entire cov matrix first, and every subplots below includes every 20 dimensions for a detailed look)

![DIAGONAL_MNIST](Screenshots/704_DIAGONAL_MNIST.png)

## Plan of the Next Week
- There are a lot of experiments and information discussed above. I want to take further digest and analysis on each of  the experiments above. I want to understand why each method excels or fails on certain datasets. 
- In addition, I want to take more time to read relevant papers and books to get more insights on the application of deep learning in anomaly detection, as well as how to optimize the performance. 
- I would be also interested to find more different dataset/generate synthetic data to test the models. 

## Appendices
### Appendix A: code to generate the Synthetic Dataset 4

In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
from random import shuffle

# Generate 100K numbers, each of which has 16 digits
# Anomaly: number of 1s is larger than 2

# Set Parameteres
n_dimensions = 16
n_samples = 10**5
data1_ratio = 0.5 # Dataset 1
data2_ratio = 0.5 # Dataset 2
Anomaly_Threshold = 4 # Anomaly if total # 1s is less than the threshold

def generate_random_mg_data(n_dimensions, n_samples):
    """
    Generate a random dataset 
    """
    mu = np.random.rand(n_dimensions) # Random vector for mean
    cov = np.random.rand(n_dimensions,n_dimensions) # Random matrix for covaraince
    data_mg = np.random.multivariate_normal(mu, cov, size=n_samples) # Generate a random matrix with multivariate normal distribution
    data_bi = data_mg >= 0.5 # Convert to binary - True if the data is larger than 0.5; otherwise 0
    data = data_bi*1 # Convert True/False to 1/0
    return data

def generate_random_data_2md(n_dimensions, data1_size, data2_size):
    """
    Generate a random data set with two multivate gaussian distribution
    """
    # Generate two dataset
    data1 = generate_random_mg_data(n_dimensions, data1_size)
    data2 = generate_random_mg_data(n_dimensions, data2_size)
    # Merge
    data = np.concatenate((data1,data2))

    # Shuffle
    shuffle(data)

    return data

np.random.seed(9001)
# Generate a random data set with two multivate gaussian distribution
data = generate_random_data_2md(n_dimensions,int(n_samples*data1_ratio),int(n_samples*data2_ratio))

# Label Anomaly if the number of 1s is less than 7
data_rowsum = np.sum(data,axis = 1)
labels = data_rowsum < Anomaly_Threshold # Anomaly if total # 1s is less than the threshold
labels = labels*1

print("Percentage of Anomaly in the dataset: " + str(np.sum(labels)/len(labels))) # Find percentage of anomaly in the dataset
print(data[labels == 1][:5]) # Print the first 5 rows of anomaly data as examples

if np.sum(labels)/len(labels) > 0.2:
    print("Too much anomaly: start cleaning!")
    labels_remove = (labels==1) & (np.random.rand(n_samples) <= 0.6) # Remove around 60% of anomalies
    print(str(sum(labels_remove)) +' Anomalies are going to be removed.')
    data = data[~labels_remove] # Remove the selected data
    labels = labels[~labels_remove] # Remove the corresponding labels
    print("Percentage of Anomaly in the dataset after cleaning: " + str(np.sum(labels)/len(labels))) # Find percentage of anomaly in the dataset


# Save the data and labels
np.save('data.npy',data)
np.save('labels.npy',labels)
print('Data and Labels have been saved!')

### Appendix B: Performance of Detectors in 4 Synthetic Datasets
#### Detector Evaluation - Reconstruction Error with PCA

|Dataset|#1|#2|#3|#4|
|---|---|---|---|---|
|Precision|16.4%|18.0%|63.7%|16.2%|
|Recall|100%|81.8%|50.4%|81.9%|
|R-Precision|18.6%|17.9%|63.7%|9.3%|
|Precision@50|12.0%|26.0%|100%|10.0%|

#### Detector Evaluation - Gaussian Models with PCA
|Dataset|#1|#2|#3|#4|
|---|---|---|---|---|
|Precision|12.9%|15.6%|13.8%|12.8%|
|Recall|100%|100.0%|86.5%|100.0%|
|R-Precision|22.9%|17.2%|63.3%|2.0%|
|Precision@50|0|22.0%|100%|0|

#### Detector Evaluation - Reconstruction Error with Autoencoder
|Dataset|#1|#2|#3|#4|
|---|---|---|---|---|
|Precision|20.2%|14.3%|85.7%|20.5%|
|Recall|82.9%|100.0%|68.3%|91.8%|
|R-Precision|14.8%|20.7%|85.7%|8.0%|
|Precision@50|4.0%|20.0%|100.0%|0|

#### Detector Evaluation - Gaussian Models with Autoencoder

|Dataset|#1|#2|#3|#4|
|---|---|---|---|---|
|Precision|71.1%|15.5%|9.9%|68.3%|
|Recall|94.5%%|99.0%|99.9%|94.0%|
|R-Precision|76.7%%|17.4%|6.1%|77.7%|
|Precision@50|100.0%|18.0%|2.0%|100.0%|